In [ ]:
#@title Install Chaterbox & Kyutai_STT
%cd /content/
## Kyutai_STT
!pip install 'sphn<0.2'
!pip install --no-deps "moshi==0.2.7"
!pip install gradio>=5.9.1
!pip install julius
!pip install librosa
!pip install soundfile

##chaterbox
!git clone https://github.com/NeuralFalconYT/chatterbox.git


def disable_watermark(file_path="./chatterbox/src/chatterbox/tts.py", disable=True):
    if not disable:
        print("Watermarking remains enabled. No changes made.")
        return

    with open(file_path, "r") as f:
        code = f.read()

    target_line = "watermarked_wav = self.watermarker.apply_watermark(wav, sample_rate=self.sr)"
    replacement_line = "watermarked_wav = wav"

    if target_line in code:
        code = code.replace(target_line, replacement_line)
        with open(file_path, "w") as f:
            f.write(code)
        print("✅ Watermarking disabled successfully.")
    else:
        print("ℹ️ Watermarking already disabled or line not found.")

def fix_chrome_hang(file_path = "./chatterbox/src/chatterbox/models/t3/t3.py"):
    try:
        with open(file_path, 'r') as f:
            content = f.read()

        content = content.replace(
            'for i in tqdm(range(max_new_tokens), desc="Sampling", dynamic_ncols=True):',
            'for i in range(max_new_tokens):'
        )

        with open(file_path, 'w') as f:
            f.write(content)

        print("✅ tqdm replaced with a normal for loop to fix Chrome hang.")
    except Exception as e:
        print(f"❌ Failed to patch file: {e}")
fix_chrome_hang()

## If you use your own voice recording and do NOT want the AI watermark,
## uncomment the following line to disable it:

# disable_watermark(disable=True)


!pip install numpy~=1.26.0
!pip install resampy==0.4.3
!pip install librosa==0.11.0
!pip install s3tokenizer
# !pip install torch==2.6.0
# !pip install torchaudio==2.6.0
!pip install transformers==4.46.3
!pip install diffusers==0.29.0
!pip install resemble-perth==1.0.1
!pip install omegaconf==2.3.0
!pip install conformer==0.3.2
!pip install safetensors==0.5.3
!pip install sentence-splitter
!pip install gradio>=5.9.1
!pip install soundfile
!pip install pysrt
from IPython.display import clear_output
clear_output()
import time
time.sleep(5)
import os
os.kill(os.getpid(), 9)

In [ ]:
#@title Mount Google Drive for auto save [ Optional, helpful for srt dubbing]
%cd /content/
from IPython.display import clear_output
from google.colab import drive
import os

auto_save_in_drive = False  # @param {type: "boolean"}

if auto_save_in_drive:
    drive.mount('/content/gdrive', force_remount=True)
    os.makedirs("/content/gdrive/MyDrive/colab_download", exist_ok=True)
    clear_output()
    print("✅ A copy of the cloned voice will be automatically saved to your Google Drive.")


In [7]:
#@title Download Model
%cd /content/chatterbox
# my useless funtions
import random
import numpy as np
import torch
import gradio as gr
import gc
import subprocess
# Initialize default pipeline
temp_audio_dir="./cloned_voices"
os.makedirs(temp_audio_dir, exist_ok=True)
def get_max_gpu_memory():
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory
        return round(total_memory / (1024 ** 3), 2)  # Convert to GB
    else:
        print("CUDA is not available.")
        return None

def is_gpu_memory_over_limit(safety_margin_gb=0.8):
    """
    Returns True if GPU memory usage exceeds (total - safety_margin).
    """
    max_memory_gb = get_max_gpu_memory()
    if max_memory_gb is None:
        return False  # Can't check memory if no GPU

    limit_gb = max_memory_gb - safety_margin_gb

    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        memory_used_mb_list = result.stdout.strip().splitlines()
        for i, memory_used_mb in enumerate(memory_used_mb_list):
            memory_used_gb = int(memory_used_mb) / 1024.0
            if memory_used_gb > limit_gb:
                print(f"Maximum GPU memory available: {max_memory_gb} GB")
                print(f"Memory limit set: {limit_gb} GB")
                print("⚠️ GPU memory usage has exceeded the safe threshold. Clearing memory and reloading the model...")
                return True
        # print("✅ GPU memory is within safe limits.")
        return False
    except Exception as e:
        print(f"Failed to check GPU memory: {e}")
        return False



from sentence_splitter import SentenceSplitter
import re
import uuid
import numpy as np
import wave
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os


def word_split(text, char_limit=300):
    words = text.split()
    chunks = []
    current_chunk = ""

    for word in words:
        if len(current_chunk) + len(word) + (1 if current_chunk else 0) <= char_limit:
            current_chunk += (" " if current_chunk else "") + word
        else:
            chunks.append(current_chunk)
            current_chunk = word

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def split_into_chunks(text, max_char_limit=300):
    if len(text)>=300:
      print("⚠️ The text is too long. Breaking it into smaller pieces so the voice generation works correctly.")

      splitter = SentenceSplitter(language='en')
      raw_sentences = splitter.split(text)

      # Flattened list of sentence-level word chunks
      sentence_chunks = []
      for sen in raw_sentences:
          sentence_chunks.extend(word_split(sen, char_limit=max_char_limit))

      chunks = []
      temp_str = ""

      for sentence in sentence_chunks:
          if len(temp_str) + len(sentence) + (1 if temp_str else 0) <= max_char_limit:
              temp_str += (" " if temp_str else "") + sentence
          else:
              chunks.append(temp_str)
              temp_str = sentence

      if temp_str:
          chunks.append(temp_str)

      return chunks
    else:
      return [text]


def clean_text(text):
    # Define replacement rules
    replacements = {
        "–": " ",  # Replace en-dash with space
        "—": " ",  #
        "-": " ",  # Replace hyphen with space
        "**": " ", # Replace double asterisks with space
        "*": " ",  # Replace single asterisk with space
        "#": " ",  # Replace hash with space
    }

    # Apply replacements
    for old, new in replacements.items():
        text = text.replace(old, new)

    # Remove emojis using regex (covering wide range of Unicode characters)
    emoji_pattern = re.compile(
        r'[\U0001F600-\U0001F64F]|'  # Emoticons
        r'[\U0001F300-\U0001F5FF]|'  # Miscellaneous symbols and pictographs
        r'[\U0001F680-\U0001F6FF]|'  # Transport and map symbols
        r'[\U0001F700-\U0001F77F]|'  # Alchemical symbols
        r'[\U0001F780-\U0001F7FF]|'  # Geometric shapes extended
        r'[\U0001F800-\U0001F8FF]|'  # Supplemental arrows-C
        r'[\U0001F900-\U0001F9FF]|'  # Supplemental symbols and pictographs
        r'[\U0001FA00-\U0001FA6F]|'  # Chess symbols
        r'[\U0001FA70-\U0001FAFF]|'  # Symbols and pictographs extended-A
        r'[\U00002702-\U000027B0]|'  # Dingbats
        r'[\U0001F1E0-\U0001F1FF]'   # Flags (iOS)
        r'', flags=re.UNICODE)

    text = emoji_pattern.sub(r'', text)

    # Remove multiple spaces and extra line breaks
    text = re.sub(r'\s+', ' ', text).strip()

    return text




def tts_file_name(text, language="en"):
    global temp_audio_dir
    # Clean and process the text
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only letters and spaces
    text = text.lower().strip().replace(" ", "_")

    # Ensure the text is not empty
    if not text:
        text = "audio"

    # Truncate to first 20 characters for filename
    truncated_text = text[:20]

    # Sanitize and format the language tag
    language = re.sub(r'\s+', '_', language.strip().lower()) if language else "unknown"

    # Generate random suffix
    random_string = uuid.uuid4().hex[:8].upper()

    # Construct the filename
    file_name = f"{temp_audio_dir}/{truncated_text}_{language}_{random_string}.wav"
    return file_name








def remove_silence_function(file_path,minimum_silence=50):
    # Extract file name and format from the provided path
    output_path = file_path.replace(".wav", "_no_silence.wav")
    audio_format = "wav"
    # Reading and splitting the audio file into chunks
    sound = AudioSegment.from_file(file_path, format=audio_format)
    audio_chunks = split_on_silence(sound,
                                    min_silence_len=100,
                                    silence_thresh=-45,
                                    keep_silence=minimum_silence)
    # Putting the file back together
    combined = AudioSegment.empty()
    for chunk in audio_chunks:
        combined += chunk
    combined.export(output_path, format=audio_format)
    return output_path

#chatterbox code
from src.chatterbox.tts import ChatterboxTTS


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)

chatterbox_model=None
def load_model():
    global chatterbox_model
    if chatterbox_model is None:
      del chatterbox_model
      gc.collect()
      torch.cuda.empty_cache()
    chatterbox_model = ChatterboxTTS.from_pretrained(DEVICE)
    return chatterbox_model

# for the first time
chatterbox_model=load_model()

import tempfile
import shutil
import os
import numpy as np
import soundfile as sf
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm


def generate(chatterbox_model, text, audio_prompt_path, exaggeration, temperature, seed_num, cfgw):
    if is_gpu_memory_over_limit():
        chatterbox_model = load_model()

    # Removed seed setting here to avoid conflicts
    # if seed_num != 0:
    #         set_seed(int(seed_num))

    wav = chatterbox_model.generate(
        text,
        audio_prompt_path=audio_prompt_path,
        exaggeration=exaggeration,
        temperature=temperature,
        cfg_weight=cfgw,
    )
    sr = chatterbox_model.sr
    audio = wav.squeeze(0).numpy()
    return sr, audio

def generate_and_save_all(text, audio_prompt_path, exaggeration, temperature, seed_num, cfgw):
    global chatterbox_model
    text = clean_text(text)
    chunks = split_into_chunks(text, max_char_limit=300)

    temp_dir = tempfile.mkdtemp(prefix="audio_chunks_")
    temp_files = []

    try:
        for idx, chunk in tqdm(enumerate(chunks), total=len(chunks), desc="Generating audio"):
            try:
                # Set seed once per chunk for reproducible variation
                if seed_num != 0:
                    set_seed(int(seed_num) + idx)

                sr, audio = generate(
                    chatterbox_model,
                    chunk,
                    audio_prompt_path,
                    exaggeration,
                    temperature,
                    seed_num=0,  # Pass 0 to skip seed setting inside generate
                    cfgw=cfgw
                )

                chunk_path = os.path.join(temp_dir, f"chunk_{idx:03}.wav")
                sf.write(chunk_path, audio, sr)
                temp_files.append(chunk_path)

            except Exception as e:
                print(f"⚠️ [Chunk {idx}] Generation failed: {e}")
                print(f"Text: {chunk}")
                print(f"Length: {len(chunk)}")
                continue  # Skip failed chunk

        # Merge all valid chunks
        final_audio = []
        for file_path in temp_files:
            try:
                data, _ = sf.read(file_path)
                final_audio.append(data)
            except Exception as e:
                print(f"💀 [Merging] Failed to read chunk: {file_path} ({e})")

        if final_audio:
            final_audio = np.concatenate(final_audio)
            final_path = tts_file_name(text, language="en")
            sf.write(final_path, final_audio, sr)
        else:
            raise RuntimeError("All audio chunk generations failed.")

    finally:
        shutil.rmtree(temp_dir)

    return final_path


from IPython.display import Audio, display

def play_audio(audio_path, autoplay=False):
    display(Audio(audio_path, autoplay=autoplay))



from IPython.display import clear_output
clear_output()
print("✅ The model has been downloaded and initialized successfully.")

✅ The model has been downloaded and initialized successfully.


In [ ]:
#@title Upload Reference Audio [Only use your own voice]
from google.colab import files
from IPython.display import clear_output

def upload_audio():
  upload_folder="/content/uploaded_audio"
  os.makedirs(upload_folder,exist_ok=True)
  os.chdir(upload_folder)
  f_names=[]
  uploaded = files.upload()
  for fn in uploaded.keys():
        f_names.append(f"{upload_folder}/{fn}")
  os.chdir("/content/chatterbox")
  clear_output()
  return f_names[-1]
uploaded_audio=upload_audio()
play_audio(uploaded_audio, autoplay=False)
uploaded_audio


In [ ]:
#@title Voice Clone
Text = "Now let's make my mum's favourite. So three mars bars into the pan. Then we add the tuna and just stir for a bit, just let the chocolate and fish infuse. A sprinkle of olive oil and some tomato ketchup. Now smell that. Oh boy this is going to be incredible."  # @param {type: "string"}

Reference_Audio = "/content/uploaded_audio/me.wav"  # @param {type: "string"}

exaggeration = 0.5  # @param {type: "number"}
cfgw = 0.5  # @param {type: "number"}
temperature = 0.8  # @param {type: "number"}
seed_num = 0  # @param {type: "number"}
Remove_Silence= False  # @param {type: "boolean"}

cloned_voice_path=generate_and_save_all(Text, Reference_Audio, exaggeration, temperature, seed_num, cfgw)
if Remove_Silence:
  cloned_voice_path=remove_silence_function(cloned_voice_path,minimum_silence=50)
from IPython.display import clear_output
clear_output()
print(f"Cloned Voice Save at {os.path.abspath(cloned_voice_path)}")
drive_folder = "/content/gdrive/MyDrive/colab_download"

if os.path.exists(drive_folder):
    drive_path = shutil.copy(cloned_voice_path, drive_folder)
    print(f"✅ Cloned voice saved to Google Drive at: {drive_path}")

# cloned_voice_path=generate_and_save_all(chatterbox_model, text, "/content/Recording (5).wav", 0.5, 0.8, 0, 0.5)


#because colab can't handle large audio to play
if len(Text)<=2000:
  play_audio(cloned_voice_path, autoplay=True)
else:
  print("📥 The audio is too long to play automatically. Please download it to listen.")


In [ ]:
#@title Download Audio

from google.colab import files
files.download(cloned_voice_path)

In [17]:
#@title Kyutai STT Subtitle
# %cd /content/
import itertools
import dataclasses
import julius
import sphn
import math
import json
import torch
import gc
import tqdm
import moshi.models


@dataclasses.dataclass
class TimestampedText:
    text: str
    timestamp: tuple[float, float]


def tokens_to_timestamped_text(text_tokens, tokenizer, frame_rate, end_of_padding_id, padding_token_id, offset_seconds):
    text_tokens = text_tokens.cpu().view(-1)
    sequence_timestamps = []

    def _tstmp(start, end):
        return (
            max(0, start / frame_rate - offset_seconds),
            max(0, end / frame_rate - offset_seconds),
        )

    def _decode(t):
        t = t[t > padding_token_id]
        return tokenizer.decode(t.numpy().tolist())

    def _decode_segment(start, end):
        start = int(start)
        end = int(end)
        text = _decode(text_tokens[start:end])
        words = text.split()
        if not words:
            return
        if len(words) == 1:
            sequence_timestamps.append(TimestampedText(text=text, timestamp=_tstmp(start, end)))
        else:
            for word in words[:-1]:
                n_tokens = len(tokenizer.encode(word))
                sequence_timestamps.append(TimestampedText(text=word, timestamp=_tstmp(start, start + n_tokens)))
                start += n_tokens
            sequence_timestamps.append(TimestampedText(text=words[-1], timestamp=_tstmp(start, end)))

    (segment_boundaries,) = torch.where(text_tokens == end_of_padding_id)
    if not segment_boundaries.numel():
        return []

    for i in range(len(segment_boundaries) - 1):
        _decode_segment(segment_boundaries[i] + 1, segment_boundaries[i + 1])

    last_start = int(segment_boundaries[-1] + 1)
    (last_end,) = torch.where(torch.isin(text_tokens[last_start:], torch.tensor([tokenizer.eos_id()])))
    last_end = last_start + (int(last_end[0]) if last_end.numel() else frame_rate)
    _decode_segment(last_start, last_end)

    return sequence_timestamps


def load_model(hf_repo="kyutai/stt-2.6b-en", device="cuda"):
    info = moshi.models.loaders.CheckpointInfo.from_hf_repo(hf_repo)
    mimi = info.get_mimi(device=device)
    moshi_model = info.get_moshi(device=device, dtype=torch.bfloat16)
    tokenizer = info.get_text_tokenizer()
    lm_gen = moshi.models.LMGen(moshi_model, temp=0, temp_text=0.0)

    return {
        "info": info,
        "mimi": mimi,
        "lm": moshi_model,
        "tokenizer": tokenizer,
        "lm_gen": lm_gen,
        "device": device
    }


def run_transcription(model, audio_path, save_json_path="word_timestamps.json"):
    info = model["info"]
    mimi = model["mimi"]
    lm_gen = model["lm_gen"]
    tokenizer = model["tokenizer"]
    device = model["device"]

    silence_prefix = info.stt_config.get("audio_silence_prefix_seconds", 1.0)
    audio_delay = info.stt_config.get("audio_delay_seconds", 5.0)
    pad_id = info.raw_config.get("text_padding_token_id", 3)

    audio, sr = sphn.read(audio_path)
    audio = torch.from_numpy(audio).to(device)
    audio = julius.resample_frac(audio, sr, mimi.sample_rate)
    if audio.shape[-1] % mimi.frame_size != 0:
        pad = mimi.frame_size - audio.shape[-1] % mimi.frame_size
        audio = torch.nn.functional.pad(audio, (0, pad))

    prefix_chunks = math.ceil(silence_prefix * mimi.frame_rate)
    suffix_chunks = math.ceil(audio_delay * mimi.frame_rate)
    silence = torch.zeros((1, 1, mimi.frame_size), dtype=torch.float32, device=device)
    chunks = itertools.chain(
        itertools.repeat(silence, prefix_chunks),
        torch.split(audio[:, None], mimi.frame_size, dim=-1),
        itertools.repeat(silence, suffix_chunks)
    )

    all_tokens = []
    with mimi.streaming(1), lm_gen.streaming(1):
        for chunk in tqdm.tqdm(chunks):
            audio_tokens = mimi.encode(chunk)
            text_tokens = lm_gen.step(audio_tokens)
            if text_tokens is not None:
                all_tokens.append(text_tokens)

    utterance_tokens = torch.concat(all_tokens, dim=-1)
    offset = prefix_chunks / mimi.frame_rate + audio_delay
    timestamped = tokens_to_timestamped_text(
        utterance_tokens,
        tokenizer,
        mimi.frame_rate,
        end_of_padding_id=0,
        padding_token_id=pad_id,
        offset_seconds=offset,
    )

    transcription_words = []
    word_timestamps = []

    for t in timestamped:
        transcription_words.append(t.text)
        word_timestamps.append({
            "word": t.text,
            "start": t.timestamp[0],
            "end": t.timestamp[1]
        })

    transcription = " ".join(transcription_words)

    if save_json_path:
        with open(save_json_path, "w", encoding="utf-8") as f:
            json.dump({
                "transcription": transcription,
                "word_timestamps": word_timestamps
            }, f, indent=2)
        # print(f"Saved: {save_json_path}")

    return transcription, word_timestamps


def unload_model(model):
    model["mimi"].to("cpu")
    model["lm"].to("cpu")
    torch.cuda.empty_cache()
    del model
    gc.collect()
    print("Model unloaded and memory freed.")

import subprocess
import soundfile as sf
import os



def ensure_mono(audio_path: str) -> str:
    """
    Ensure an audio file is mono. If stereo, converts to mono using FFmpeg.
    Returns the path to the mono audio file (original or new).
    Raises RuntimeError if conversion fails.
    """
    try:
        with sf.SoundFile(audio_path) as f:
            channels = f.channels

        if channels == 1:
            # print(f"✅ Audio is already mono: {audio_path}")
            return audio_path

        # print(f"⚠️ Audio is stereo (channels={channels}), converting to mono MP3...")

        base = os.path.splitext(os.path.basename(audio_path))[0]

        mono_path = f"./subtitle/{base}_mono.mp3"

        # Remove stale output if exists
        if os.path.exists(mono_path):
            os.remove(mono_path)

        # Run FFmpeg conversion
        command = [
            "ffmpeg", "-i", audio_path,
            "-ac", "1",
            "-y", mono_path
        ]
        subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Validate conversion success
        if not os.path.isfile(mono_path) or os.path.getsize(mono_path) == 0:
            raise RuntimeError(f"FFmpeg failed: '{mono_path}' not created or is empty.")

        # print(f"✅ Converted to mono MP3: {mono_path}")
        return mono_path

    except Exception as e:
        raise RuntimeError(f"Mono conversion failed for '{audio_path}': {e}") from e

import os
import subprocess

def convert_video_to_mono_mp3(video_path):
    # Create output path by replacing extension
    base=os.path.splitext(os.path.basename(video_path))[0]
    mono_mp3_path = "./subtitle/"+base + "_mono.mp3"
    if os.path.exists(mono_mp3_path):
        os.remove(mono_mp3_path)
    # FFmpeg command
    command = [
        "ffmpeg", "-i", video_path,   # Input video file
        "-vn",                        # Remove video
        "-ac", "1",                   # Mono audio
        "-ar", "44100",               # Optional: Set sample rate
        "-y",                         # Overwrite output
        mono_mp3_path
    ]

    # Run command silently
    subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # print("✅ Saved to:", mono_mp3_path)
    return mono_mp3_path




import string

def write_word_srt(word_level_timestamps, output_file="word.srt", skip_punctuation=True):
    with open(output_file, "w", encoding="utf-8") as f:
        index = 1  # Track subtitle numbering separately

        for entry in word_level_timestamps:
            word = entry["word"]

            # Skip punctuation if enabled
            if skip_punctuation and all(char in string.punctuation for char in word):
                continue

            start_time = entry["start"]
            end_time = entry["end"]

            # Convert seconds to SRT time format (HH:MM:SS,mmm)
            def format_srt_time(seconds):
                hours = int(seconds // 3600)
                minutes = int((seconds % 3600) // 60)
                sec = int(seconds % 60)
                millisec = int((seconds % 1) * 1000)
                return f"{hours:02}:{minutes:02}:{sec:02},{millisec:03}"

            start_srt = format_srt_time(start_time)
            end_srt = format_srt_time(end_time)

            # Write entry to SRT file
            f.write(f"{index}\n{start_srt} --> {end_srt}\n{word}\n\n")
            index += 1  # Increment subtitle number




def split_line_by_char_limit(text, max_chars_per_line=38):
    """
    Formats a text block into lines with a maximum character limit.
    Returns a list of strings (the lines).
    """
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if not current_line:
            current_line = word
        elif len(current_line + " " + word) <= max_chars_per_line:
            current_line += " " + word
        else:
            lines.append(current_line)
            current_line = word
    if current_line:
        lines.append(current_line)
    return lines

import re


def merge_punctuation_glitches(subtitles):
    """
    Adjusts punctuation at subtitle boundaries.
    - Moves punctuation from the start of a subtitle to the end of the previous one.
    - Removes awkward solo punctuation subtitles.
    - Removes all quotes and unwanted characters cleanly.
    """
    cleaned = [subtitles[0]]
    for i in range(1, len(subtitles)):
        prev = cleaned[-1]
        curr = subtitles[i]

        prev_text = prev["text"].rstrip()
        curr_text = curr["text"].lstrip()

        # 1. Move leading punctuation from current to previous
        match = re.match(r'^([,.:;!?]+)(\s*)(.+)', curr_text)
        if match:
            punct, _, rest = match.groups()
            if not prev_text.endswith(punct):
                prev["text"] = prev_text + punct
            curr_text = rest.strip()

        # 2. Clean unwanted characters like quotes, colons, semicolons
        unwanted_chars = ['"', '“', '”', ';', ':']
        for ch in unwanted_chars:
            curr_text = curr_text.replace(ch, '')
        curr_text = curr_text.strip()

        # 3. Skip subtitle if it’s just punctuation or now empty
        if not curr_text or re.fullmatch(r'[.,!?]+', curr_text):
            # Extend end time of previous to cover this orphaned punctuation
            prev["end"] = curr["end"]
            continue

        # 4. Apply cleaned text back
        curr["text"] = curr_text
        prev["text"] = prev["text"].replace('"', '').replace('“', '').replace('”', '')

        cleaned.append(curr)

    return cleaned




def write_sentence_srt(
    word_level_timestamps,
    output_file="subtitles_professional.srt",
    max_lines=2,
    max_duration_s=7.0,
    max_chars_per_line=38,
    hard_pause_threshold=0.5,
    merge_pause_threshold=0.4 # NEW: Threshold for merging single-word orphans
):
    """
    Creates professional-grade SRT files using a two-phase process:
    1. Generates visually-aware subtitles.
    2. Performs a post-processing pass to merge single-word orphans.
    """
    if not word_level_timestamps:
        return

    # --- PHASE 1: Generate good "draft" subtitles ---
    draft_subtitles = []
    i = 0
    while i < len(word_level_timestamps):
        start_time = word_level_timestamps[i]["start"]
        current_words = []

        j = i
        while j < len(word_level_timestamps):
            entry = word_level_timestamps[j]
            potential_words = current_words + [entry["word"]]
            potential_text = " ".join(potential_words)

            # Check hard limits before adding the word
            if len(split_line_by_char_limit(potential_text, max_chars_per_line)) > max_lines: break
            if (entry["end"] - start_time) > max_duration_s and current_words: break

            if j > i:
                prev_entry = word_level_timestamps[j-1]
                pause = entry["start"] - prev_entry["end"]
                if pause >= hard_pause_threshold: break
                if prev_entry["word"].endswith(('.','!','?')): break

            current_words.append(entry["word"])
            j += 1

        if not current_words:
            current_words.append(word_level_timestamps[i]["word"])
            j = i + 1

        text = " ".join(current_words)
        end_time = word_level_timestamps[j - 1]["end"]
        draft_subtitles.append({ "start": start_time, "end": end_time, "text": text })
        i = j

    # --- PHASE 2: Post-processing to merge single-word orphans ---
    if not draft_subtitles:
        return

    final_subtitles = [draft_subtitles[0]]
    for k in range(1, len(draft_subtitles)):
        prev_sub = final_subtitles[-1]
        current_sub = draft_subtitles[k]

        # Check if current subtitle is a single-word orphan
        is_single_word_orphan = len(current_sub["text"].split()) == 1
        pause_from_prev = current_sub["start"] - prev_sub["end"]

        if is_single_word_orphan and pause_from_prev < merge_pause_threshold:
            # Attempt to merge it backwards
            merged_text = prev_sub["text"] + " " + current_sub["text"]

            # Only merge if it doesn't violate the line limit
            if len(split_line_by_char_limit(merged_text, max_chars_per_line)) <= max_lines:
                # Success! Update the previous subtitle instead of adding a new one.
                prev_sub["text"] = merged_text
                prev_sub["end"] = current_sub["end"]
                continue # Skip adding the current_sub as it's been merged

        # If not merged, just add the current subtitle as is
        final_subtitles.append(current_sub)
    final_subtitles = merge_punctuation_glitches(final_subtitles)
    # --- Write the final, polished SRT file ---
    def format_srt_time(seconds):
        h, m, s, ms = int(seconds//3600), int((seconds%3600)//60), int(seconds%60), int((seconds%1)*1000)
        return f"{h:02}:{m:02}:{s:02},{ms:03}"

    with open(output_file, "w", encoding="utf-8") as f:
        for idx, sub_data in enumerate(final_subtitles, start=1):
            text = sub_data["text"].replace(" ,", ",").replace(" .", ".")
            formatted_lines = split_line_by_char_limit(text, max_chars_per_line)
            f.write(f"{idx}\n")
            f.write(f"{format_srt_time(sub_data['start'])} --> {format_srt_time(sub_data['end'])}\n")
            f.write("\n".join(formatted_lines) + "\n\n")

    # print(f"SRT file '{output_file}' created with orphan-merging logic.")




def srt_making(upload_file, model_name="kyutai/stt-2.6b-en"):
    root_path = "./subtitle"
    os.makedirs(root_path, exist_ok=True)

    model = load_model(hf_repo=model_name, device="cuda")

    if upload_file.endswith(".mp4"):
        mono_audio_path = convert_video_to_mono_mp3(upload_file)
    else:
        mono_audio_path = ensure_mono(upload_file)

    base = os.path.splitext(os.path.basename(upload_file))[0]
    save_json_path = f"{root_path}/{base}_word.json"
    word_srt_path = f"{root_path}/{base}_word.srt"
    sentence_srt_path = f"{root_path}/{base}_sentence.srt"
    text_path = f"{root_path}/{base}_text.txt"

    transcription, word_level_timestamps = run_transcription(
        model, mono_audio_path, save_json_path
    )

    unload_model(model)

    write_word_srt(word_level_timestamps, output_file=word_srt_path, skip_punctuation=True)
    write_sentence_srt(word_level_timestamps, output_file=sentence_srt_path)

    with open(text_path, "w", encoding="utf-8") as f:
        f.write(transcription)

    return sentence_srt_path, word_srt_path, save_json_path, text_path, transcription





##################################
audio_or_video_path = '/content/test.mp3'  # @param {type: "string"}
choose_model = "kyutai/stt-2.6b-en"  # @param ["kyutai/stt-1b-en_fr", "kyutai/stt-2.6b-en"]

sentence_srt_path,word_srt_path,save_json_path,text_path,transcription=srt_making(audio_or_video_path,choose_model)
sentence_srt_path=os.path.abspath(sentence_srt_path)
from IPython.display import clear_output
clear_output()
# print(f"Sentence SRT path: {sentence_srt_path}")
# print(f"Word SRT path: {word_srt_path}")
# print(f"JSON path: {save_json_path}")
# print(f"Text path: {text_path}")
# print(f"Transcription: {transcription}")

sentence_srt_path

Sentence SRT path: ./subtitle/test_sentence.srt


In [ ]:
#@title Download Subtitle

from google.colab import files
files.download(sentence_srt_path)



## 📘 English-to-English Dubbing Workflow (Fix Accent, Grammar, and Pronunciation)

This process helps correct bad English speaking, improve grammar, remove filler words, and enhance pronunciation  while **keeping subtitle timestamps unchanged**.

### 🧾 Steps:

1. **Download the subtitle file** (`.srt` format).
2. **Open the file using Notepad** and **copy all the subtitle text**.
3. **Paste the text into [Gemini](https://gemini.google.com/)** or any AI editor.
4. Instruct Gemini:

   > “Fix grammar, pronunciation, and filler words. If there is bad English, improve it while preserving the original meaning. **Do not change any timestamps**.”
5. Once Gemini returns the fixed version, **copy the edited subtitle data**.
6. Open **Notepad**, paste the corrected subtitle text, and **save it with a `.srt` extension** (e.g., `fixed_subtitles.srt`).
7. **Upload the new SRT file to your Colab environment**.
8. Use the **corrected subtitle file** for your **SRT-to-audio dubbing pipeline**.




In [ ]:
#@title Upload  fixed english .SRT file
from google.colab import files
from IPython.display import clear_output

def upload_srt():
  upload_folder="/content/uploaded_srt"
  os.makedirs(upload_folder,exist_ok=True)
  os.chdir(upload_folder)
  f_names=[]
  uploaded = files.upload()
  for fn in uploaded.keys():
        f_names.append(f"{upload_folder}/{fn}")
  os.chdir("/content/chatterbox")
  clear_output()
  return f_names[-1]
uploaded_srt=upload_srt()
uploaded_srt

In [20]:
#@title subtitle
import os
import re
import uuid
import shutil
import platform
import datetime
import subprocess
import math
import json
import pysrt
import librosa
import soundfile as sf
from tqdm.auto import tqdm
from pydub import AudioSegment

# ---------------------- Utility Functions ----------------------

# Returns the current time formatted as HH_MM_AM/PM (for filenames or logs)
def get_current_time():
    return datetime.datetime.now().strftime("%I_%M_%p")

# Constructs an output file path for the final dubbed audio
def get_subtitle_Dub_path(srt_file_path, lang="en"):
    file_name = os.path.splitext(os.path.basename(srt_file_path))[0]
    full_base_path = os.path.join(os.getcwd(), "TTS_DUB")
    os.makedirs(full_base_path, exist_ok=True)
    random_string = str(uuid.uuid4())[:6]
    new_path = os.path.join(full_base_path, f"{file_name}_{lang}_{random_string}.wav")
    return new_path.replace("__", "_")

# Removes noise characters like [♫] from the subtitle text and saves a cleaned SRT
def clean_srt(input_path):
    def clean_srt_line(text):
        for bad in ["[", "]", "♫"]:
            text = text.replace(bad, "")
        return text.strip()

    subs = pysrt.open(input_path, encoding='utf-8')
    output_path = input_path.lower().replace(".srt", "") + "_.srt"
    with open(output_path, "w", encoding='utf-8') as file:
        for sub in subs:
            file.write(f"{sub.index}\n{sub.start} --> {sub.end}\n{clean_srt_line(sub.text)}\n\n")
    return output_path





# Checks if FFmpeg is available on the system; if not, warns user and returns fallback
# To change audio speed explicitly, we can use either FFmpeg or Librosa.
def is_ffmpeg_installed():
    ffmpeg_exe = "ffmpeg.exe" if platform.system() == "Windows" else "ffmpeg"
    try:
        subprocess.run([ffmpeg_exe, "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
        return True, ffmpeg_exe
    except Exception:
        gr.Warning("FFmpeg not found. Falling back to librosa for audio speedup.", duration=20)
        return False, ffmpeg_exe

# Because FFmpeg can handle speeds from 0.5× to 2.0× only
def atempo_chain(factor):
    if 0.5 <= factor <= 2.0:
        return f"atempo={factor:.3f}"
    parts = []
    while factor > 2.0:
        parts.append("atempo=2.0")
        factor /= 2.0
    while factor < 0.5:
        parts.append("atempo=0.5")
        factor *= 2.0
    parts.append(f"atempo={factor:.3f}")
    return ",".join(parts)

# If FFmpeg is not found, we will use Librosa
def speedup_audio_librosa(input_file, output_file, speedup_factor):
    try:
        y, sr = librosa.load(input_file, sr=None)
        y_stretched = librosa.effects.time_stretch(y, rate=speedup_factor)
        sf.write(output_file, y_stretched, sr)
    except Exception as e:
        gr.Warning(f"Librosa speedup failed: {e}")
        shutil.copy(input_file, output_file)

# Change the audio speed if it exceeds the original SRT segment duration.
def change_speed(input_file, output_file, speedup_factor, use_ffmpeg, ffmpeg_path):
    if use_ffmpeg:
        try:
            subprocess.run(
                [ffmpeg_path, "-i", input_file, "-filter:a", atempo_chain(speedup_factor), output_file, "-y"],
                check=True,
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
        except Exception as e:
            gr.Error(f"FFmpeg speedup error: {e}")
            speedup_audio_librosa(input_file, output_file, speedup_factor)
    else:
        speedup_audio_librosa(input_file, output_file, speedup_factor)

# Remove silence from the start and end of the audio.
def remove_edge_silence(input_path, output_path):
    y, sr = librosa.load(input_path, sr=None)
    trimmed_audio, _ = librosa.effects.trim(y, top_db=30)
    sf.write(output_path, trimmed_audio, sr)
    return output_path


def remove_silence_function(file_path,minimum_silence=50):
    # Extract file name and format from the provided path
    output_path = file_path.replace(".wav", "_no_silence.wav")
    audio_format = "wav"
    # Reading and splitting the audio file into chunks
    sound = AudioSegment.from_file(file_path, format=audio_format)
    audio_chunks = split_on_silence(sound,
                                    min_silence_len=100,
                                    silence_thresh=-45,
                                    keep_silence=minimum_silence)
    # Putting the file back together
    combined = AudioSegment.empty()
    for chunk in audio_chunks:
        combined += chunk
    combined.export(output_path, format=audio_format)
    return output_path
from IPython.display import clear_output

# ---------------------- Main Class ----------------------
class SRTDubbing:
    def __init__(self, use_ffmpeg=True, ffmpeg_path="ffmpeg"):
        self.use_ffmpeg = use_ffmpeg
        self.ffmpeg_path = ffmpeg_path
        self.cache_dir = "./cache"
        os.makedirs("./dummy", exist_ok=True)
        os.makedirs(self.cache_dir, exist_ok=True)

    @staticmethod
    # Because our target is single-speaker SRT dubbing,
    # we will calculate the speaker's average talking speed per second.
    def get_avg_speaker_speed(srt_path):
        subs = pysrt.open(srt_path, encoding='utf-8')
        speeds = []
        for sub in subs:
            duration_sec = (sub.end.ordinal - sub.start.ordinal) / 1000
            char_count = len(sub.text.replace(" ", ""))
            if duration_sec > 0 and char_count > 0:
                speeds.append(char_count / duration_sec)
        return sum(speeds) / len(speeds) if speeds else 14

    @staticmethod
    # Calculate the speaker's default talking speed (e.g., 0.5x, 1x, 1.5x)
    def get_speed_factor(srt_path, default_tts_rate=14):
        avg_rate = SRTDubbing.get_avg_speaker_speed(srt_path)
        speed_factor = avg_rate / default_tts_rate if default_tts_rate > 0 else 1.0
        return math.floor(speed_factor * 100) / 100  # Truncate

    @staticmethod
    # Merge multiple SRT segments if the gap is small and total duration
    # stays under N milliseconds
    def merge_fast_entries(entries, max_pause_gap=1000, max_merged_duration_ms=8000):
        merged = []
        i = 0
        n = len(entries)
        while i < n:
            curr = entries[i].copy()
            j = i + 1
            while j < n:
                next_ = entries[j]
                gap = next_["start_time"] - curr["end_time"]
                new_duration = next_["end_time"] - curr["start_time"]
                if gap > max_pause_gap or new_duration > max_merged_duration_ms:
                    break
                if not curr["text"].strip().endswith((".", "!", "?")):
                    curr["text"] = curr["text"].strip() + ","
                curr["text"] += " " + next_["text"]
                curr["end_time"] = next_["end_time"]
                j += 1
            merged.append(curr)
            i = j
        return merged

    @staticmethod
    # Convert SRT timestamp to milliseconds
    def convert_to_millisecond(t):
        return t.hours * 3600000 + t.minutes * 60000 + t.seconds * 1000 + int(t.milliseconds)

    # Read SRT file and convert it to our required dictionary format for dubbing
    def read_srt_file(self, file_path):
        subs = pysrt.open(file_path, encoding='utf-8')
        entries = []
        prev_end = 0
        for idx, sub in enumerate(subs, 1):
            start = self.convert_to_millisecond(sub.start)
            end = self.convert_to_millisecond(sub.end)
            pause = start - prev_end if idx > 1 else start
            entries.append({
                'entry_number': idx,
                'start_time': start,
                'end_time': end,
                'text': sub.text.strip(),
                'pause_time': pause,
                'audio_name': f"{idx}.wav",
                'previous_pause': f"{idx}_before_pause.wav",
            })
            prev_end = end

        entries = self.merge_fast_entries(entries)

        ## For debug
        # with open("./old.json", "w", encoding="utf-8") as f:
        #     json.dump(entries, f, indent=2, ensure_ascii=False)
        # with open("/content/new.json", "w", encoding="utf-8") as f:
        #     json.dump(entries, f, indent=2, ensure_ascii=False)

        return entries


    # For TTS, modify this function in the future to use a different TTS or voice cloning tool
    def text_to_speech_srt(self, text, audio_path, reference_audio, exaggeration,temperature,seed,cfgw,actual_duration, default_speed_factor=None):
        import soundfile as sf
        from librosa import get_duration

        TOLERANCE_MS = 30
        temp = os.path.join(self.cache_dir, "temp.wav")

        if default_speed_factor is None:
            default_speed_factor = 1.0
        # Step 1: create clone voice
        path=generate_and_save_all(text, reference_audio, exaggeration, temperature, seed, cfgw)

        # Step 2: Apply user-defined speaking speed
        if default_speed_factor != 1.0:
            user_speed_path = path.replace(".wav", "_user.wav")
            change_speed(path, user_speed_path, default_speed_factor, self.use_ffmpeg, self.ffmpeg_path)
            path = user_speed_path

        # Step 3: Trim silence
        remove_edge_silence(path, temp)

        # Step 4: Duration analysis (high precision)
        y, sr = sf.read(temp)
        duration_ms = int(get_duration(y=y, sr=sr) * 1000)

        # Step 5: If very close, skip correction
        if abs(duration_ms - actual_duration) <= TOLERANCE_MS:
            shutil.move(temp, audio_path)
            return

        # Step 6: Try regenerating with silence removal if too long
        if duration_ms > actual_duration:
            path=remove_silence_function(temp,minimum_silence=50)
            shutil.copy(path, temp)
            y, sr = sf.read(temp)
            duration_ms = int(get_duration(y=y, sr=sr) * 1000)

        # Step 7: Final correction
        if duration_ms > actual_duration + TOLERANCE_MS:
            factor = duration_ms / actual_duration
            corrected = os.path.join(self.cache_dir, "speed_final.wav")
            change_speed(temp, corrected, factor, self.use_ffmpeg, self.ffmpeg_path)
            shutil.move(corrected, audio_path)
        elif duration_ms < actual_duration - TOLERANCE_MS:
            silence = AudioSegment.silent(duration=actual_duration - duration_ms)
            (AudioSegment.from_file(temp) + silence).export(audio_path, format="wav")
        else:
            shutil.move(temp, audio_path)


    @staticmethod
    # Insert silent gaps between two segments
    def make_silence(duration, path):
        AudioSegment.silent(duration=duration).export(path, format="wav")

    @staticmethod
    # Srt save folder
    def create_folder_for_srt(srt_file_path):
        base = os.path.splitext(os.path.basename(srt_file_path))[0]
        folder = f"./dummy/{base}_{str(uuid.uuid4())[:4]}"
        os.makedirs(folder, exist_ok=True)
        return folder

    @staticmethod
    # Join Chunks audio files
    def concatenate_audio_files(paths, output):
        audio = sum([AudioSegment.from_file(p) for p in paths], AudioSegment.silent(duration=0))
        audio.export(output, format="wav")

    # Util funtion to call other funtions
    def srt_to_dub(self, srt_path, output_path, reference_audio, exaggeration,temperature,seed,cfgw,speaker_talk_speed=True):
        entries = self.read_srt_file(srt_path)
        folder = self.create_folder_for_srt(srt_path)
        all_audio = []
        if speaker_talk_speed:
          default_speed_factor = self.get_speed_factor(srt_path)
        else:
          default_speed_factor=1.0
        total=len(entries)
        count=1
        for entry in tqdm(entries):
            print(f"{count}/{total}",end="\r")
            count+=1
            self.make_silence(entry['pause_time'], os.path.join(folder, entry['previous_pause']))
            all_audio.append(os.path.join(folder, entry['previous_pause']))
            tts_path = os.path.join(folder, entry['audio_name'])
            self.text_to_speech_srt(entry['text'], tts_path, reference_audio, exaggeration,temperature,seed,cfgw, entry['end_time'] - entry['start_time'], default_speed_factor)
            all_audio.append(tts_path)
            clear_output()
        self.concatenate_audio_files(all_audio, output_path)

# ---------------------- Entrypoint ----------------------
def dubbing(srt_path, reference_audio, exaggeration=1.0, temperature=0.7, seed=0, cfgw=3.0,speaker_talk_speed=True):
    if not srt_path.endswith(".srt"):
        gr.Error("Please upload a valid .srt file", duration=5)
        return None

    use_ffmpeg, ffmpeg_path = is_ffmpeg_installed()
    processed_srt =  clean_srt(srt_path)
    output_path = get_subtitle_Dub_path(srt_path)

    SRTDubbing(use_ffmpeg, ffmpeg_path).srt_to_dub(processed_srt, output_path, reference_audio, exaggeration,temperature,seed,cfgw,speaker_talk_speed)
    return output_path


# -------------------------------
# Example Usage
# -------------------------------

srt_path = "/content/chatterbox/subtitle/test_sentence.srt" # @param {type: "string"}
reference_audio = "/content/uploaded_audio/me.wav"# @param {type: "string"}
exaggeration = 0.5 # @param {type: "number"}
cfgw = 0.5 # @param {type: "number"}
temperature = 0.8 # @param {type: "number"}
seed = 0 # @param {type: "number"}
speaker_talk_speed= 0 # @param {type: "boolean"}
result_path =  dubbing(srt_path, reference_audio, exaggeration, temperature, seed, cfgw,speaker_talk_speed)
clear_output()
print(f"Dubbed audio saved at:\n{result_path}")

drive_folder = "/content/gdrive/MyDrive/colab_download"
if os.path.exists(drive_folder):
    drive_result_path = shutil.copy(result_path, drive_folder)
    print(f"Copied to Google Drive:\n{drive_result_path}")






Dubbed audio saved at:
/content/chatterbox/TTS_DUB/test_sentence_en_e9bc12.wav


In [21]:
#@title Download Dubbing Audio

from google.colab import files
files.download(result_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Play Dubbing Audio [Avoid large audio on colab]
play_audio(result_path)



## 🎬 Tips for English-to-English Video Dubbing and Editing

For **cloning voice from a video**, it's best to **first extract the `.mp3` audio** from your video and then upload it to **Google Colab** for subtitle generation and dubbing. This reduces upload time significantly.

---

### 🧪 Silence Removal Strategy:

* If your video is a **tutorial** and needs **accurate subtitle-aligned dubbing**, first **remove silence from the video**, then extract the `.mp3` file.
* Use [`auto-editor`](https://pypi.org/project/auto-editor/) — a Python tool to automatically remove silence from videos.

```bash
# Example command
auto-editor input.mp4 --silent-threshold 0.03 --export-as mp4
```

---

### 🛠️ Basic Video Editing Steps:

* Use **any video editing software** (e.g., **CapCut**).
* **Mute the original video audio** .
* **Add the cloned voice audio** .
* Optionally, **add subtitles** for accessibility and clarity.

---

### 🗣️ For Vlogs and Self-Talk Videos:

* You can edit the full video first.
* Then, at the end, **remove silence** using `auto-editor` to improve flow.
---

### 🎯 Goal:

* Original audio should be [Kyutai STT give wrong timestamp when there is music]:

  * Free of background noise
  * Clear and slow speech
  * Easy to process
* The dubbing system will **fix grammar**, **accent**, and **English pronunciation**, while **preserving the meaning and natural tone**.

<br>

Yes, if we use lip sync, there's no need to remove silence that's super cool. However, some open-source models perform lip sync with poor quality on low-end GPUs. High-quality or upscaled videos require even more GPU power, RAM, and processing time. For longer audio (e.g., 5–10 minutes) at 25–30 FPS, the processing time increases significantly. And after all that, Google Colab often crashes, which ends up being a total waste of time. So, lip sync is great if you have your own GPU. Colab really struggles when handling large video files.